# Подгружаем все что может понадобится

In [41]:
import pandas as pd
import numpy as np
import sys
import datetime
import matplotlib.pyplot as plt
from sortedcontainers import SortedDict
import warnings
import math
from copy import deepcopy
from tqdm import tqdm
import matplotlib as mpl
import platform
from collections import Counter
from IPython.core.display import HTML
import os
from copy import deepcopy
import json
from copy import deepcopy
from catboost import CatBoostClassifier
from statistics import median
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier



warnings.filterwarnings('ignore')
pd.options.display.max_rows = 10000




In [12]:
#%pip install --user lightautoml

# Посмотрим на наши данные и почистим их

In [13]:
#fol = pd.read_csv("/Users/a185583357/Downloads/MMzip/fol/LKOH.txt", sep=",")

#TODO Inspect 20150923 - lots of errors there
fol = pd.read_csv("/Users/a185583357/Desktop/MOEX/OrderLog20150310/OrderLog20150310.txt", sep=',')
fol["TRADENO"] = fol["TRADENO"].astype(str)
fol.tail()

,NO,SECCODE,BUYSELL,TIME,ORDERNO,ACTION,PRICE,VOLUME,TRADENO,TRADEPRICE
26214948,26214949,FXIT,S,184457675,12836191,0,2461.0000,11285,nan,NaN
26214949,26214950,OGKB,S,184458220,12805810,0,0.2164,1000,nan,NaN
26214950,26214951,SBER,B,184458269,12819204,0,69.0900,30,nan,NaN
26214951,26214952,VZRZP,S,184458683,6479754,0,91.5000,20,nan,NaN
26214952,26214953,FXGD,S,184459634,12836206,0,519.0000,56500,nan,NaN


In [14]:


count = Counter(fol["SECCODE"]).most_common(30)
out = []
for share in count:
    out.append(share[0])
count

[('GAZP', 2997383),
 ('SBER', 2290044),
 ('SNGS', 2087074),
 ('CHMF', 1425078),
 ('RTKM', 1339749),
 ('LKOH', 1049386),
 ('ROSN', 844480),
 ('SBERP', 738394),
 ('VTBR', 714055),
 ('HYDR', 713783),
 ('SNGSP', 712352),
 ('MGNT', 704247),
 ('TRNFP', 694222),
 ('GMKN', 631399),
 ('NLMK', 593623),
 ('MTSS', 593605),
 ('MTLR', 581235),
 ('MOEX', 551809),
 ('URKA', 468585),
 ('TATN', 459544),
 ('ALRS', 456695),
 ('MAGN', 436832),
 ('EONR', 418226),
 ('FEES', 377768),
 ('AFKS', 366013),
 ('IRAO', 330446),
 ('MVID', 258997),
 ('NVTK', 251434),
 ('OGKB', 191821),
 ('SIBN', 191810)]

In [15]:
fol.head(5)

,NO,SECCODE,BUYSELL,TIME,ORDERNO,ACTION,PRICE,VOLUME,TRADENO,TRADEPRICE
0,1,ENRU,B,100000000,1,1,0.785,100000,nan,NaN
1,2,SBERP,B,100000000,2,1,52.400,10000,nan,NaN
2,3,AFKS,B,100000000,3,1,15.880,100,nan,NaN
3,4,AFLT,B,100000000,4,1,37.000,100,nan,NaN
4,5,ALRS,S,100000000,5,1,77.000,20000,nan,NaN


#  Соберем стакан из этого

Общая идея взята из статьи - https://habr.com/ru/post/276635/

In [16]:
from sortedcontainers import SortedDict

class DOM:
    def __init__(self):
        self.bid = SortedDict() #descending
        self.ask = SortedDict()
        self.bid_active_order = dict()
        self.ask_active_order = dict()
        self.day = None
        self.flag = None
        
    
    def check_order(self, data):
        if self.day != data["DATE"]:
            self.clean_dom()
            self.day = data["DATE"]
        
        if data["ACTION"] == 1:
            self.check_add(data)
        elif data["ACTION"] == 0:
            self.delete_order(data)
        else:
            self.trade(data)
        
    ## First part - adding order
        
    def check_add(self, data):
        if data["PRICE"] < 0.000000001:
            self.add_order(data)
            pass
        else:
            self.add_order(data)
                
    
    def add_order(self, data):
        if data['BUYSELL'] == "B":
            self.bid_active_order[data["ORDERNO"]] = (data["PRICE"], data["VOLUME"])
            if data["PRICE"] not in self.bid:
                self.bid[data["PRICE"]] = data["VOLUME"]
            else:
                self.bid[data["PRICE"]] += data["VOLUME"]
        else:
            self.ask_active_order[data["ORDERNO"]] = (data["PRICE"], data["VOLUME"])
            if data["PRICE"] not in self.ask:
                self.ask[data["PRICE"]] = data["VOLUME"]
            else:
                self.ask[data["PRICE"]] += data["VOLUME"]
    ## Second part - delete order
    
    def delete_order(self, data):
        if data['BUYSELL'] == "B":
            self.bid[data["PRICE"]] -= self.bid_active_order[data["ORDERNO"]][1]
            del self.bid_active_order[data["ORDERNO"]]
            if self.bid[data["PRICE"]] < 0.000000001:
                del self.bid[data["PRICE"]]
        else:
            self.ask[data["PRICE"]] -= self.ask_active_order[data["ORDERNO"]][1]
            del self.ask_active_order[data["ORDERNO"]]
            if self.ask[data["PRICE"]] < 0.000000001:
                del self.ask[data["PRICE"]]
    
    ## Third part - match order   
    def trade(self, data):
        if data['BUYSELL'] == "B":
            self.exec_buy_order(data)
        else:
            self.exec_sell_order(data)
    

    def exec_sell_order(self, data):
        self.flag = None
        order = self.ask_active_order[data["ORDERNO"]]
        if data["VOLUME"] < order[1]:
            self.ask_active_order[data["ORDERNO"]] = (data["PRICE"], 
                                                      order[1] - data["VOLUME"])
        elif data["VOLUME"] == order[1]:
            del self.ask_active_order[data["ORDERNO"]]
        else:
            #iseberg or error
            #raise Exception(self.ask_active_order[data["ORDERNO"]], data)
            del self.ask_active_order[data["ORDERNO"]]
            self.flag = data["ORDERNO"]
            
        
        if data["PRICE"] not in self.ask:
            raise Exception
            pass
        else:
            self.ask[data["PRICE"]] -= min(order[1], data["VOLUME"])
            if self.ask[data["PRICE"]] < 0.000000001:
                del self.ask[data["PRICE"]]

    
    
    def exec_buy_order(self, data):
        self.flag = None
        order = self.bid_active_order[data["ORDERNO"]]
        if data["VOLUME"] < order[1]:
            self.bid_active_order[data["ORDERNO"]] = (data["PRICE"], 
                                                      order[1] - data["VOLUME"])
        elif data["VOLUME"] == order[1]:
            del self.bid_active_order[data["ORDERNO"]]
        else:
            #iseberg or error
            #raise Exception(self.bid_active_order[data["ORDERNO"]], data)
            del self.bid_active_order[data["ORDERNO"]]
            flag = data["ORDERNO"]
        
        if data["PRICE"] not in self.bid:
            raise Exception(data, self.bid)
            pass
        else:
            self.bid[data["PRICE"]] -= min(order[1], data["VOLUME"])
            if self.bid[data["PRICE"]] < 0.000000001:
                del self.bid[data["PRICE"]]
    
    ## Others
    
        
    def clean_dom(self):
        self.__init__()
        
        

In [17]:
date = "20150310"
data = fol[fol["SECCODE"] == "SBER"]
data = data[data["ACTION"] != 2]
data["DATE"] = date
records = data.to_dict("records")
data.reset_index(inplace=True)
data.shape

(2115840, 12)

In [18]:
dom = DOM()
errors = 0
isebergs = pd.read_csv("/Users/a185583357/Desktop/MOEX/OrderLog" + date + "/isebergs" + date + ".csv", index_col=0).to_dict()["vol"]

for i in tqdm(range(len(records))):
    record = records[i].copy()
    if record["ACTION"] == 1 and record["ORDERNO"] in isebergs:
        record["VOLUME"] = isebergs[record["ORDERNO"]]
    try:
        out = dom.check_order(record)
    except Exception as e:
        print(e)
        if record["ACTION"] == 1:
            print("MAYBE Some problem, cant add order")
            dom.check_order(record)
            raise Exception
        errors += 1

100%|██████████| 2115840/2115840 [00:04<00:00, 437059.42it/s]


# Создадим стакан способный сам матчить все заявки и сравним с оригиналом

In [19]:
date = "20150310"
data = fol[fol["SECCODE"] == "SBER"]
data = data[data["ACTION"] != 2]
data["DATE"] = date
records = data.to_dict("records")
data.reset_index(inplace=True)
data.shape

(2115840, 12)

In [20]:
data.head()

,index,NO,SECCODE,BUYSELL,TIME,ORDERNO,ACTION,PRICE,VOLUME,TRADENO,TRADEPRICE,DATE
0,47,48,SBER,B,100000000,48,1,70.00,20,nan,NaN,20150310
1,49,50,SBER,B,100000000,50,1,70.50,880,nan,NaN,20150310
2,94,95,SBER,B,100000000,95,1,72.45,15000,nan,NaN,20150310
3,138,139,SBER,B,100000000,139,1,72.23,10700,nan,NaN,20150310
4,147,148,SBER,B,100000000,148,1,71.00,100,nan,NaN,20150310


In [21]:
class Matching_Engine:
    def __init__(self):
        self.bid = SortedDict()  # descending
        self.ask = SortedDict()
        self.bid_active_order = dict()
        self.ask_active_order = dict()
        self.bid_FIFO = dict()
        self.ask_FIFO = dict()
        self.day = None
        self.share_name = "LKOH"
        self.trade_log = []

    def check_order(self, data):
        if self.day != data["DATE"]:
            self.clean_dom()
            self.share_name = data["SECCODE"]
            self.day = data["DATE"]

        if data["ACTION"] == 1:
            return self.check_add(data, new_trades=[])
        elif data["ACTION"] == 2:
            pass
        else:
            self.delete_order(data)

    ## First part - adding order

    def check_add(self, data, rec_flag=False, new_trades=[]):
        if (len(self.ask) > 0 and data['BUYSELL'] == "B" and \
            (data["PRICE"] >= self.ask.peekitem(0)[0] or data["PRICE"] < 0.000001)) or \
                (len(self.bid) > 0 and data['BUYSELL'] == "S" and data["PRICE"] <= self.bid.peekitem(-1)[0]):
            if not rec_flag:
                self.add_order(data)
            return self.trade(data, new_trades=new_trades)
        else:
            if not rec_flag:
                self.add_order(data)

    def add_order(self, data):
        if data['BUYSELL'] == "B":
            self.bid_active_order[data["ORDERNO"]] = (data["PRICE"], data["VOLUME"])
            if data["PRICE"] not in self.bid:
                self.bid[data["PRICE"]] = data["VOLUME"]
                self.bid_FIFO[data["PRICE"]] = [data["ORDERNO"]]
            else:
                self.bid[data["PRICE"]] += data["VOLUME"]
                self.bid_FIFO[data["PRICE"]].append(data["ORDERNO"])
        else:
            self.ask_active_order[data["ORDERNO"]] = (data["PRICE"], data["VOLUME"])
            if data["PRICE"] not in self.ask:
                self.ask[data["PRICE"]] = data["VOLUME"]
                self.ask_FIFO[data["PRICE"]] = [data["ORDERNO"]]
            else:
                self.ask[data["PRICE"]] += data["VOLUME"]
                self.ask_FIFO[data["PRICE"]].append(data["ORDERNO"])

    ## Second part - delete order

    def delete_order(self, data):
        if data['BUYSELL'] == "B":
            self.bid_FIFO[data["PRICE"]].pop(self.bid_FIFO[data["PRICE"]].index(data["ORDERNO"]))
            self.bid[data["PRICE"]] -= self.bid_active_order[data["ORDERNO"]][1]
            if len(self.bid_FIFO[data["PRICE"]]) < 1:
                del self.bid_FIFO[data["PRICE"]]
            del self.bid_active_order[data["ORDERNO"]]
            if self.bid[data["PRICE"]] < 0.000000001:
                del self.bid[data["PRICE"]]
        else:
            self.ask[data["PRICE"]] -= self.ask_active_order[data["ORDERNO"]][1]
            self.ask_FIFO[data["PRICE"]].pop(self.ask_FIFO[data["PRICE"]].index(data["ORDERNO"]))
            if len(self.ask_FIFO[data["PRICE"]]) < 1:
                del self.ask_FIFO[data["PRICE"]]
            del self.ask_active_order[data["ORDERNO"]]
            if self.ask[data["PRICE"]] < 0.000000001:
                del self.ask[data["PRICE"]]

    ## Third part - match order
    def trade(self, data, new_trades=[]):
        if data['BUYSELL'] == "B":
            return self.exec_buy_order(data, new_trades=new_trades)
        else:
            return self.exec_sell_order(data, new_trades=new_trades)

    def exec_sell_order(self, data, new_trades=[]):
        go_trade= True
        while go_trade:
            go_trade = False
            sell_order = self.ask_active_order[data["ORDERNO"]]
            tradeprice = self.bid.peekitem(-1)[0]
            order_no = self.bid_FIFO[tradeprice][0]
            if str(order_no)[0] == 'm':
                out_no = "mm_past"
            else:
                out_no = order_no
            buy_order = self.bid_active_order[order_no]
            if buy_order[1] > sell_order[1]:
                self.bid_active_order[order_no] = (self.bid_active_order[order_no][0],
                                                   buy_order[1] - sell_order[1])
                trade_vol = sell_order[1]
                self.bid[buy_order[0]] -= trade_vol
                self.delete_order(data)
            elif buy_order[1] == sell_order[1]:
                self.delete_order(data)
                tmp = dict()
                tmp["BUYSELL"] = "B"

                tmp["PRICE"] = buy_order[0]
                tmp["ORDERNO"] = order_no

                self.delete_order(tmp)
                trade_vol = sell_order[1]
            else:
                # iseberg or error
                tmp = dict()
                tmp["BUYSELL"] = "B"
                tmp["PRICE"] = buy_order[0]
                tmp["ORDERNO"] = order_no
                self.delete_order(tmp)
                trade_vol = buy_order[1]
                self.ask[sell_order[0]] -= trade_vol
                self.ask_active_order[data["ORDERNO"]] = (self.ask_active_order[data["ORDERNO"]][0],
                                                          sell_order[1] - buy_order[1])
                # TODO не учтено полное пробитие стакана!!
                if self.bid.peekitem(-1)[0] >= sell_order[0] or data["PRICE"] < 0.000000001:
                    go_trade = True

            self.trade_log.append(
                [tradeprice, trade_vol, data["ORDERNO"], out_no, tradeprice * trade_vol, data["TIME"], data["NO"], "S"])
            new_trades.append(self.trade_log[-1])
            if buy_order[1] < 0 or sell_order[1] < 0:
                raise Exception(buy_order, sell_order)
            if go_trade:
                data["VOLUME"] -= trade_vol
                self.ask_active_order[data["ORDERNO"]] = (self.ask_active_order[data["ORDERNO"]][0], data["VOLUME"])
                # self.ask[data["PRICE"]] -= data["VOLUME"]
                #return self.check_add(data, rec_flag=True, new_trades=new_trades)
        return new_trades

    def exec_buy_order(self, data, new_trades=[]):
        go_trade = True
        while go_trade:
            go_trade = False
            buy_order = self.bid_active_order[data["ORDERNO"]]
            tradeprice = self.ask.peekitem(0)[0]
            order_no = self.ask_FIFO[tradeprice][0]
            if str(order_no)[0] == 'm':
                out_no = "mm_past"
            else:
                out_no = order_no
            sell_order = self.ask_active_order[order_no]
            if sell_order[1] > buy_order[1]:
                self.ask_active_order[order_no] = (self.ask_active_order[order_no][0],
                                                   sell_order[1] - buy_order[1])
                trade_vol = buy_order[1]
                self.ask[sell_order[0]] -= trade_vol
                self.delete_order(data)
            elif buy_order[1] == sell_order[1]:
                self.delete_order(data)
                tmp = dict()
                tmp["BUYSELL"] = "S"
                tmp["PRICE"] = sell_order[0]
                tmp["ORDERNO"] = self.ask_FIFO[tradeprice][0]
                self.delete_order(tmp)
                trade_vol = sell_order[1]
            else:
                # iseberg or error
                tmp = dict()
                tmp["BUYSELL"] = "S"
                tmp["PRICE"] = sell_order[0]
                tmp["ORDERNO"] = order_no
                self.delete_order(tmp)
                trade_vol = sell_order[1]
                self.bid[buy_order[0]] -= trade_vol
                self.bid_active_order[data["ORDERNO"]] = (self.bid_active_order[data["ORDERNO"]][0],
                                                          buy_order[1] - sell_order[1])
                if self.ask.peekitem(0)[0] <= buy_order[0] or data["PRICE"] < 0.000000001:
                    go_trade = True

            # TODO Мне кажется здесь потенциальная ошибка, не предусмотрено что аркет мейкер заявка может в кого то удариться
            self.trade_log.append(
                [tradeprice, trade_vol, data["ORDERNO"], out_no, tradeprice * trade_vol, data["TIME"], data["NO"], "B"])
            new_trades.append(self.trade_log[-1])
            if buy_order[1] < 0 or sell_order[1] < 0:
                raise Exception(buy_order, sell_order)
            if go_trade:
                data["VOLUME"] -= trade_vol
                self.bid_active_order[data["ORDERNO"]] = (self.bid_active_order[data["ORDERNO"]][0], data["VOLUME"])
                # self.bid[data["PRICE"]] -= data["VOLUME"]
                return self.check_add(data, rec_flag=True, new_trades=new_trades)
        return new_trades

    ## Others

    def clean_dom(self):
        self.__init__()



In [22]:
dom = Matching_Engine()
errors = 0
isebergs = pd.read_csv("/Users/a185583357/Desktop/MOEX/OrderLog" + date + "/isebergs" + date + ".csv", index_col=0).to_dict()["vol"]

for i in tqdm(range(len(records))):
    record = records[i].copy()
    if record["ACTION"] == 1 and record["ORDERNO"] in isebergs:
        record["VOLUME"] = isebergs[record["ORDERNO"]]
    try:
        out = dom.check_order(record)
    except Exception as e:
        #print(e)
        if record["ACTION"] == 1:
            print("MAYBE Some problem, cant add order")
            dom.check_order(record)
            raise Exception
        errors += 1

100%|██████████| 2115840/2115840 [00:08<00:00, 242653.55it/s]


In [23]:
dom.trade_log[-1]

[69.97, 100, 12836092, 12836086, 6997.0, 183958906, 26213007, 'B']

# Теперь сделаем заготовку для ML, для этого соберем numpy array снапшота стакана и последих трейдов

In [24]:
deep = 1

In [153]:
def RSI (data, time_window):
    diff = data.diff(1).dropna()        # diff in one field(one day)

    #this preservers dimensions off diff values
    up_chg = deepcopy(0 * diff)
    down_chg = deepcopy(0 * diff)
    
    # up change is equal to the positive difference, otherwise equal to zero
    up_chg[diff > 0] = diff[ diff>0 ]
    
    # down change is equal to negative deifference, otherwise equal to zero
    down_chg[diff < 0] = diff[ diff < 0 ]
   
    
    # check pandas documentation for ewm
    # https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html
    # values are related to exponential decay
    # we set com=time_window-1 so we get decay alpha=1/time_window
    
    up_chg_avg   = up_chg.mean()
    down_chg_avg = down_chg.mean()
    
    if down_chg_avg == 0:
        return 100.0
    rs = abs(up_chg_avg/down_chg_avg)
    rsi = 100 - 100/(1+rs)
    return rsi

In [154]:
def VWAP(trade_log, record, vwap, top_price, min_price, timerange=1):
    timenow = datetime.datetime.strptime(str(record["TIME"]), '%H%M%S%f')
    time_log = pd.to_datetime(pd.Series(np.array(trade_log)[:, -3]), format='%H%M%S%f')
    trades_window = np.array(trade_log)[time_log > (timenow - datetime.timedelta(seconds=timerange))]
    if trades_window.shape[0] == 0:
        return vwap, top_price, min_price
    else:
        return sum(trades_window[:, 0].astype(float) * trades_window[:, 1].astype(int)) /  sum(trades_window[:, 1].astype(int)), max(trades_window[:, 0].astype(float)), min(trades_window[:, 0].astype(float))

In [155]:
tmp = datetime.datetime.strptime(str(record["TIME"]), '%H%M%S%f')
time_log = pd.to_datetime(pd.Series(np.array(dom.trade_log)[:, -3]), format='%H%M%S%f')
tmp = np.array(dom.trade_log)[time_log > (tmp - datetime.timedelta(seconds=500))]
max(tmp[:, 0].astype(float))

69.98

In [156]:
sma5, sma10, sma30, sma50, sma100 = 0, 0, 0, 0, 0
ema5, ema10, ema30, ema100 = 0, 0, 0, 0
std10, std50, std100, std1000 = 0, 0, 0, 0
rsi14, rsi50, rsi100, rsi300 = 0, 0, 0, 0
vwap1, vwap30, vwap60 = 0, 0, 0
top_price1, top_price30, top_price60 = 0, 0, 0
min_price1, min_price30, min_price60 = 0, 0, 0

def get_technical_indicator(change, trade_log, output, record):
    #TODO это однозначно можно сделать оптимальнее!!
    global deep
    global sma5, sma10, sma30, sma50, sma100
    global ema5, ema10, ema30, ema100
    global std10, std50, std100, std1000 
    global rsi14, rsi50, rsi100, rsi300
    global vwap1, vwap30, vwap60
    global top_price1, top_price30, top_price60
    global min_price1, min_price30, min_price60
    if change != 0:        
        sma5 = np.array(trade_log[-5:])[:, 0].astype(float).mean()
        sma10 = np.array(trade_log[-10:])[:, 0].astype(float).mean()
        sma30 = np.array(trade_log[-30:])[:, 0].astype(float).mean()
        sma50 = np.array(trade_log[-50:])[:, 0].astype(float).mean()
        sma100 = np.array(trade_log[-100:])[:, 0].astype(float).mean()
        #EMA
        ema5 = pd.DataFrame(np.array(trade_log[-5:])[:, 0].astype(float)).ewm(span=5).mean().iat[-1, 0]
        ema10 = pd.DataFrame(np.array(trade_log[-10:])[:, 0].astype(float)).ewm(span=10).mean().iat[-1, 0]
        ema30 = pd.DataFrame(np.array(trade_log[-30:])[:, 0].astype(float)).ewm(span=30).mean().iat[-1, 0]
        ema100 = pd.DataFrame(np.array(trade_log[-100:])[:, 0].astype(float)).ewm(span=30).mean().iat[-1, 0]
        #std
        std10 = np.array(trade_log[-10:])[:, 0].astype(float).std()
        std50 = np.array(trade_log[-50:])[:, 0].astype(float).std()
        std100 = np.array(trade_log[-100:])[:, 0].astype(float).std()
        std1000 = np.array(trade_log[-1000:])[:, 0].astype(float).std()
        #RSI
        rsi14 = RSI(pd.Series(np.array(trade_log[-14:])[:, 0].astype(float)), 14)
        rsi50 = RSI(pd.Series(np.array(trade_log[-50:])[:, 0].astype(float)), 50)
        rsi100 = RSI(pd.Series(np.array(trade_log[-100:])[:, 0].astype(float)), 100)
        rsi300 = RSI(pd.Series(np.array(trade_log[-300:])[:, 0].astype(float)), 300)
        #VWAP, top, min
        vwap1, top_price1, min_price1 = VWAP(trade_log[-1000:], record, vwap1, top_price1, min_price1, timerange=1)
        vwap30, top_price30, min_price30 = VWAP(trade_log[-3000:], record, vwap30, top_price30, min_price60, timerange=30)
        vwap60, top_price60, min_price60 = VWAP(trade_log[-5000:], record, vwap60, top_price30, min_price60, timerange=60)
    output[deep * 4 + 7] = sma5
    output[deep * 4 + 8] = sma10
    output[deep * 4 + 9] = sma30
    output[deep * 4 + 10] = sma50
    output[deep * 4 + 11] = sma100
    #EMA
    output[deep * 4 + 12] = ema5
    output[deep * 4 + 13] = ema10
    output[deep * 4 + 14] = ema30
    output[deep * 4 + 15] = ema100
    #std
    output[deep * 4 + 16] = std10
    output[deep * 4 + 17] = std50
    output[deep * 4 + 18] = std100
    output[deep * 4 + 19] = std1000
    #RSI
    output[deep * 4 + 20] = rsi14
    output[deep * 4 + 21] = rsi50
    output[deep * 4 + 22] = rsi100
    output[deep * 4 + 23] = rsi300
    #VWAP
    output[deep * 4 + 24] = vwap1
    output[deep * 4 + 25] = vwap30
    output[deep * 4 + 26] = vwap60
    #top
    output[deep * 4 + 27] = top_price1
    output[deep * 4 + 28] = top_price30
    output[deep * 4 + 29] = top_price60
    #min
    output[deep * 4 + 30] = min_price1
    output[deep * 4 + 31] = min_price30
    output[deep * 4 + 32] = min_price60
    return output

In [157]:
def construct_raw(dom, deep, raw_data, record):
    output = [0] * (deep * 4 + 33)
    trade_log = dom.trade_log
    bids = list(dom.bid.items())
    asks = list(dom.ask.items())
    for i in range(0, deep):
        if len(bids) > i:
            #print(bids[-1], i)
            output[(2 * deep) - 2 * i - 1] = bids[-i - 1][1]
            output[(2 * deep) - 2 * i - 2] = bids[-i - 1][0]
        if len(asks) > i:
            #print(asks, i)
            output[(2 * deep) + 2 * i + 1] = asks[i][1]
            output[(2 * deep) + 2 * i] = asks[i][0]
    if len(dom.trade_log) > 0:
        #print(dom.trade_log[-1])
        output[deep * 4] = dom.trade_log[-1][0]
        output[deep * 4 + 1] = dom.trade_log[-1][1]
        if output[deep * 4 + 2] == "B":
            output[deep * 4 + 2] = -1
        else:
            output[deep * 4 + 2] = 1
        change = raw_data[-1][deep * 4] - output[deep * 4]
        if change == 0:
            output[deep * 4 + 3] = 0
        elif change > 0:
            output[deep * 4 + 3] = -1
        else:
            output[deep * 4 + 3] = 1
            
        output[deep * 4 + 4] = dom.trade_log[-1][-3]
    output[deep * 4 + 5] = int(record["DATE"])
    output[deep * 4 + 6] = record["TIME"]
    if len(trade_log) > 50:
        output = get_technical_indicator(change, trade_log, output, record)
    
    
    return output

In [ ]:
def get_data(date, records, deep):
    dom = Matching_Engine()
    errors = 0
    isebergs = pd.read_csv("/Users/a185583357/Desktop/MOEX/OrderLog" + date + "/isebergs" + date + ".csv", index_col=0).to_dict()["vol"]
    raw_data = []

    for i in tqdm(range(len(records))):
        record = records[i].copy()
        if record["ACTION"] == 1 and record["ORDERNO"] in isebergs:
            record["VOLUME"] = isebergs[record["ORDERNO"]]
        try:
            out = dom.check_order(record)
            raw_data.append(construct_raw(dom, deep, raw_data, record))
        except Exception as e:
            #print(e)
            if record["ACTION"] == 1:
                print("MAYBE Some problem, cant add order")
                dom.check_order(record)
                raise Exception
            errors += 1
    return raw_data


raw_data = get_data(date, records, deep)
#raw_data[1000]

 36%|███▋      | 771226/2115840 [13:26<46:12, 485.05it/s]  

In [ ]:
def create_future_vwap(snapshot):
    snapchot["FutureVWAP1"] = 0 * snapchot["EMA5"]
    snapchot["FutureVWAP30"] = 0 * snapchot["EMA5"]
    snapchot["FutureVWAP60"] = 0 * snapchot["EMA5"]
    

In [ ]:
def get_df_y(raw_data, deep):
    columns = []
    for col in ["Bid", "Ask"]:
        for num in range(1, 1 + deep):
            if col == "Bid":
                num = deep - num + 1
            columns.append(col + "Price" + str(num))
            columns.append(col + "Volume" + str(num))
    dupl_col = deepcopy(columns)
    columns += ["Last Trade Price"] + ["Last Trade Volume"] + ["Side"] + ["isNewTrade"] + \
    ["LastDealTime"] + ["Date"] + ["Time"] + ["SMA5"] + ["SMA10"] + ["SMA30"] + ["SMA50"] + ["SMA100"] +\
    ["EMA5"] + ["EMA10"] + ["EMA30"] + ["EMA100"] + ["STD10"] + ["STD50"] + ["STD100"] + ["STD1000"] +\
    ["RSI14"] + ["RSI50"] + ["RSI100"] + ["RSI300"] + ["VWAP1"] + ["VWAP30"] + ["VWAP60"] +\
    ["TOP1"] + ["TOP30"] + ["TOP60"] + ["MIN1"] + ["MIN30"] + ["MIN60"] 
    snapschot = pd.DataFrame(np.stack(raw_data, axis=0), columns=columns)
    snapschot["LiquidityBid"] = 0
    snapschot["LiquidityAsk"] = 0
    for i in range(1, 1 + deep):
        snapschot['LiquidityAsk'] += snapschot["AskVolume" + str(i)]
        snapschot["LiquidityBid"] += snapschot["BidVolume" + str(i)]

    snapschot["BidLiquidityRatio"] = snapschot["LiquidityBid"] / (snapschot["LiquidityBid"] + snapschot["LiquidityAsk"])
    snapschot = snapschot[snapschot["Time"] > 100002000.0]
    snapschot.drop_duplicates(subset=dupl_col,inplace=True)
    snapschot["Time"] = pd.to_datetime((snapschot["Time"]), format='%H%M%S%f')
    y = np.array(snapschot["isNewTrade"])
    df = snapschot.drop(["isNewTrade"], axis=1)
    return np.array(df), np.array(y), snapschot

df, y, snapschot = get_df_y(raw_data, deep)
y
snapschot.to_csv("SBER20150310.csv")

## Attention надо обязательно сделать задержку и смещение

Данные готовы, по одному дню сбера. теперь посмотрим на них

## Здесь будут графики и анализ существующих данных

In [ ]:
snapschot[30000:30050]

In [ ]:
visu_data = snapschot#[1500:]

In [ ]:
plt.scatter(visu_data["Time"], visu_data["Last Trade Price"])

In [ ]:
pd.to_datetime((snapschot["Time"]), format='%H%M%S%f')

In [33]:
from sklearn.model_selection import TimeSeriesSplit

tscv = TimeSeriesSplit(3) # мало сплитов потому что по некоторым спутникам слишком мало наблюдений
for train_index, test_index in tscv.split(df):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = df[train_index], df[test_index]
    y_train, y_test = y[train_index], y[test_index]

TRAIN: [    0     1     2 ... 30646 30647 30648] TEST: [30649 30650 30651 ... 61292 61293 61294]
TRAIN: [    0     1     2 ... 61292 61293 61294] TEST: [61295 61296 61297 ... 91938 91939 91940]
TRAIN: [    0     1     2 ... 91938 91939 91940] TEST: [ 91941  91942  91943 ... 122584 122585 122586]


In [36]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)

RandomForestClassifier()

In [37]:
Counter(rf.predict(X_test))

Counter({0.0: 29086, -1.0: 1341, 1.0: 219})

In [49]:
cbst = CatBoostClassifier()
cbst.fit(pd.DataFrame(X_train), y_train)


Learning rate set to 0.099647
0:	learn: 0.9963072	total: 41.9ms	remaining: 41.9s
1:	learn: 0.9193130	total: 65.8ms	remaining: 32.8s
2:	learn: 0.8581060	total: 85.4ms	remaining: 28.4s
3:	learn: 0.8094343	total: 105ms	remaining: 26.2s
4:	learn: 0.7705729	total: 126ms	remaining: 25.2s
5:	learn: 0.7378676	total: 147ms	remaining: 24.3s
6:	learn: 0.7109435	total: 166ms	remaining: 23.5s
7:	learn: 0.6880398	total: 185ms	remaining: 23s
8:	learn: 0.6689561	total: 209ms	remaining: 23s
9:	learn: 0.6536122	total: 233ms	remaining: 23.1s
10:	learn: 0.6409128	total: 256ms	remaining: 23s
11:	learn: 0.6294693	total: 278ms	remaining: 22.9s
12:	learn: 0.6195174	total: 300ms	remaining: 22.8s
13:	learn: 0.6110921	total: 320ms	remaining: 22.6s
14:	learn: 0.6041051	total: 340ms	remaining: 22.3s
15:	learn: 0.5978718	total: 360ms	remaining: 22.1s
16:	learn: 0.5930573	total: 378ms	remaining: 21.8s
17:	learn: 0.5883358	total: 396ms	remaining: 21.6s
18:	learn: 0.5844062	total: 418ms	remaining: 21.6s
19:	learn: 0.5

170:	learn: 0.5249642	total: 3.56s	remaining: 17.3s
171:	learn: 0.5248731	total: 3.58s	remaining: 17.3s
172:	learn: 0.5247408	total: 3.6s	remaining: 17.2s
173:	learn: 0.5246548	total: 3.62s	remaining: 17.2s
174:	learn: 0.5245313	total: 3.64s	remaining: 17.2s
175:	learn: 0.5243385	total: 3.66s	remaining: 17.1s
176:	learn: 0.5241625	total: 3.68s	remaining: 17.1s
177:	learn: 0.5239652	total: 3.7s	remaining: 17.1s
178:	learn: 0.5238948	total: 3.72s	remaining: 17.1s
179:	learn: 0.5238310	total: 3.73s	remaining: 17s
180:	learn: 0.5236607	total: 3.75s	remaining: 17s
181:	learn: 0.5235712	total: 3.77s	remaining: 17s
182:	learn: 0.5234347	total: 3.79s	remaining: 16.9s
183:	learn: 0.5233168	total: 3.82s	remaining: 16.9s
184:	learn: 0.5232482	total: 3.83s	remaining: 16.9s
185:	learn: 0.5231131	total: 3.85s	remaining: 16.9s
186:	learn: 0.5229750	total: 3.87s	remaining: 16.8s
187:	learn: 0.5228799	total: 3.89s	remaining: 16.8s
188:	learn: 0.5228063	total: 3.91s	remaining: 16.8s
189:	learn: 0.522722

334:	learn: 0.5087541	total: 6.71s	remaining: 13.3s
335:	learn: 0.5086967	total: 6.73s	remaining: 13.3s
336:	learn: 0.5086076	total: 6.76s	remaining: 13.3s
337:	learn: 0.5084938	total: 6.77s	remaining: 13.3s
338:	learn: 0.5084173	total: 6.79s	remaining: 13.2s
339:	learn: 0.5083692	total: 6.81s	remaining: 13.2s
340:	learn: 0.5082949	total: 6.83s	remaining: 13.2s
341:	learn: 0.5082143	total: 6.84s	remaining: 13.2s
342:	learn: 0.5081300	total: 6.86s	remaining: 13.1s
343:	learn: 0.5080638	total: 6.88s	remaining: 13.1s
344:	learn: 0.5079797	total: 6.9s	remaining: 13.1s
345:	learn: 0.5079101	total: 6.92s	remaining: 13.1s
346:	learn: 0.5078729	total: 6.94s	remaining: 13.1s
347:	learn: 0.5077459	total: 6.96s	remaining: 13s
348:	learn: 0.5076274	total: 6.98s	remaining: 13s
349:	learn: 0.5075385	total: 7s	remaining: 13s
350:	learn: 0.5074388	total: 7.02s	remaining: 13s
351:	learn: 0.5073655	total: 7.04s	remaining: 13s
352:	learn: 0.5071645	total: 7.06s	remaining: 12.9s
353:	learn: 0.5070814	tota

499:	learn: 0.4961043	total: 9.88s	remaining: 9.88s
500:	learn: 0.4959946	total: 9.9s	remaining: 9.86s
501:	learn: 0.4959316	total: 9.92s	remaining: 9.84s
502:	learn: 0.4958554	total: 9.94s	remaining: 9.82s
503:	learn: 0.4957817	total: 9.96s	remaining: 9.8s
504:	learn: 0.4957036	total: 9.98s	remaining: 9.78s
505:	learn: 0.4956525	total: 10s	remaining: 9.76s
506:	learn: 0.4955879	total: 10s	remaining: 9.74s
507:	learn: 0.4954839	total: 10s	remaining: 9.72s
508:	learn: 0.4954251	total: 10.1s	remaining: 9.7s
509:	learn: 0.4953865	total: 10.1s	remaining: 9.67s
510:	learn: 0.4953453	total: 10.1s	remaining: 9.65s
511:	learn: 0.4952656	total: 10.1s	remaining: 9.64s
512:	learn: 0.4951980	total: 10.1s	remaining: 9.62s
513:	learn: 0.4951393	total: 10.1s	remaining: 9.6s
514:	learn: 0.4950494	total: 10.2s	remaining: 9.57s
515:	learn: 0.4949939	total: 10.2s	remaining: 9.55s
516:	learn: 0.4948908	total: 10.2s	remaining: 9.53s
517:	learn: 0.4948045	total: 10.2s	remaining: 9.51s
518:	learn: 0.4947127	

663:	learn: 0.4855099	total: 13s	remaining: 6.59s
664:	learn: 0.4854444	total: 13.1s	remaining: 6.58s
665:	learn: 0.4853748	total: 13.1s	remaining: 6.56s
666:	learn: 0.4852959	total: 13.1s	remaining: 6.54s
667:	learn: 0.4852639	total: 13.1s	remaining: 6.52s
668:	learn: 0.4852055	total: 13.1s	remaining: 6.5s
669:	learn: 0.4851775	total: 13.1s	remaining: 6.47s
670:	learn: 0.4851310	total: 13.2s	remaining: 6.45s
671:	learn: 0.4851119	total: 13.2s	remaining: 6.43s
672:	learn: 0.4850654	total: 13.2s	remaining: 6.41s
673:	learn: 0.4849958	total: 13.2s	remaining: 6.39s
674:	learn: 0.4849447	total: 13.2s	remaining: 6.37s
675:	learn: 0.4848802	total: 13.3s	remaining: 6.36s
676:	learn: 0.4848104	total: 13.3s	remaining: 6.34s
677:	learn: 0.4847220	total: 13.3s	remaining: 6.32s
678:	learn: 0.4846187	total: 13.3s	remaining: 6.3s
679:	learn: 0.4845047	total: 13.3s	remaining: 6.28s
680:	learn: 0.4844418	total: 13.4s	remaining: 6.26s
681:	learn: 0.4843758	total: 13.4s	remaining: 6.24s
682:	learn: 0.48

825:	learn: 0.4765280	total: 16.2s	remaining: 3.41s
826:	learn: 0.4764725	total: 16.2s	remaining: 3.39s
827:	learn: 0.4764280	total: 16.2s	remaining: 3.37s
828:	learn: 0.4763843	total: 16.3s	remaining: 3.35s
829:	learn: 0.4763327	total: 16.3s	remaining: 3.34s
830:	learn: 0.4762616	total: 16.3s	remaining: 3.32s
831:	learn: 0.4762204	total: 16.3s	remaining: 3.3s
832:	learn: 0.4761987	total: 16.3s	remaining: 3.28s
833:	learn: 0.4761469	total: 16.4s	remaining: 3.26s
834:	learn: 0.4760744	total: 16.4s	remaining: 3.24s
835:	learn: 0.4760197	total: 16.4s	remaining: 3.22s
836:	learn: 0.4759672	total: 16.4s	remaining: 3.2s
837:	learn: 0.4759220	total: 16.5s	remaining: 3.18s
838:	learn: 0.4758862	total: 16.5s	remaining: 3.16s
839:	learn: 0.4758292	total: 16.5s	remaining: 3.15s
840:	learn: 0.4757944	total: 16.5s	remaining: 3.13s
841:	learn: 0.4757541	total: 16.6s	remaining: 3.11s
842:	learn: 0.4757153	total: 16.6s	remaining: 3.09s
843:	learn: 0.4756947	total: 16.6s	remaining: 3.07s
844:	learn: 0.

987:	learn: 0.4687782	total: 19.8s	remaining: 241ms
988:	learn: 0.4686946	total: 19.8s	remaining: 221ms
989:	learn: 0.4686519	total: 19.8s	remaining: 201ms
990:	learn: 0.4686138	total: 19.9s	remaining: 180ms
991:	learn: 0.4685536	total: 19.9s	remaining: 160ms
992:	learn: 0.4685113	total: 19.9s	remaining: 140ms
993:	learn: 0.4684749	total: 19.9s	remaining: 120ms
994:	learn: 0.4684427	total: 19.9s	remaining: 100ms
995:	learn: 0.4683845	total: 20s	remaining: 80.2ms
996:	learn: 0.4683565	total: 20s	remaining: 60.1ms
997:	learn: 0.4683348	total: 20s	remaining: 40.1ms
998:	learn: 0.4682584	total: 20s	remaining: 20.1ms
999:	learn: 0.4682093	total: 20.1s	remaining: 0us


In [70]:
Counter(cbst.predict(X_test).ravel())

Counter({0.0: 29765, -1.0: 793, 1.0: 88})

In [71]:
Counter(y_test)

Counter({0.0: 25369, 1.0: 2623, -1.0: 2654})

In [51]:
X_train.shape

(91941, 30)